In [1]:
# Ruta
RUTA = '/content/drive/MyDrive/Universidad/Evapotranspiration-Prediction-2022/data_processing/'

In [2]:
import pandas as pd
import datetime
import pytz

In [ ]:
dataset = pd.read_csv(RUTA+'data/Custom_location_-33_3896_-70_794525_62c2dd3b055760000724ffd5.csv')

In [ ]:
dataset_new = dataset[["forecast dt unixtime","forecast dt iso","slice dt unixtime","slice dt iso","temperature","wind_speed","humidity","clouds"]]

In [ ]:
def date_to_local(date):
  "date: string"
  import pytz
  fecha_prueba = date
  fecha_dt = datetime.datetime.strptime(fecha_prueba, '%Y-%m-%d %H:%M:%S %z %Z')
  fecha_local = fecha_dt.astimezone(pytz.timezone('America/Santiago'))
  fecha_local_str = str(fecha_local.year)+'-'+str(fecha_local.month)+'-'+str(fecha_local.day)
  return fecha_local_str

In [ ]:
for i in range(len(dataset_new)):
  dataset_new['forecast dt iso'][i] = date_to_local(dataset_new['forecast dt iso'][i])
  dataset_new['slice dt iso'][i] = date_to_local(dataset_new['slice dt iso'][i])
  print('iteración: ',i)

In [ ]:
dataset_new['slice dt iso'] = pd.to_datetime(dataset_new['slice dt iso'],format='%Y-%m-%d')
dataset_new['forecast dt iso'] = pd.to_datetime(dataset_new['forecast dt iso'],format='%Y-%m-%d')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
dataset_new['forecast_slice'] = 0

In [ ]:
# df_prueba.groupby(['slice dt iso'])[['temperature']].min()
dataset_new.to_csv(RUTA+'data/forecast_data_openweathermap_01012021-31052022.csv')

In [ ]:
datos = pd.read_csv(RUTA+'data/forecast_data_openweathermap_01012021-31052022.csv')

In [ ]:
datos['forecast_slice'] = datos['forecast dt iso'] +' '+ datos['slice dt iso']

In [ ]:
datos_new = pd.DataFrame()
datos_new['t_min'] = datos.groupby(['forecast_slice'])[['temperature']].min()
datos_new['t_max'] = datos.groupby(['forecast_slice'])[['temperature']].max()
datos_new['temp'] = datos.groupby(['forecast_slice'])[['temperature']].mean()
datos_new['wind'] = datos.groupby(['forecast_slice'])[['wind_speed']].mean()
datos_new['humidity_min'] = datos.groupby(['forecast_slice'])[['humidity']].min()
datos_new['humidity_max'] = datos.groupby(['forecast_slice'])[['humidity']].max()
datos_new['humidity'] = datos.groupby(['forecast_slice'])[['humidity']].mean()
datos_new['clouds'] = datos.groupby(['forecast_slice'])[['clouds']].mean()

In [ ]:
forecast = []
f_slice = []
for i in range(len(datos_new.index)):
  forecast.append(datos_new.index[i][:10]) #Forecast date
  f_slice.append(datos_new.index[i][11:]) #Forecast slice

In [ ]:
datos_new['forecast date'] = forecast
datos_new['slice date'] = f_slice

In [ ]:
datos_new.to_csv(RUTA+'forecast_historical/reduced_forecast_data_openweathermap_01012021-31052022.csv')

In [7]:
df = pd.read_csv(RUTA+'forecast_historical/reduced_forecast_data_openweathermap_01012021-31052022.csv')

In [17]:
fechas = df['forecast date'].unique()

In [40]:
df.where(df['forecast date']== '2020-12-31').dropna()[1:8]

,forecast_slice,t_min,t_max,temp,wind,humidity_min,humidity_max,humidity,clouds,forecast date,slice date
1,2020-12-31 2021-01-01,10.25,31.28,19.287917,2.782083,21.45,79.99,44.003333,37.916667,2020-12-31,2021-01-01
2,2020-12-31 2021-01-02,9.18,29.80,18.692917,2.660417,26.30,87.93,57.930000,0.000000,2020-12-31,2021-01-02
3,2020-12-31 2021-01-03,9.29,32.36,19.245417,2.787500,20.10,91.20,54.326667,0.000000,2020-12-31,2021-01-03
4,2020-12-31 2021-01-04,10.14,32.08,19.446667,2.754583,22.70,84.89,52.485833,0.000000,2020-12-31,2021-01-04
5,2020-12-31 2021-01-05,7.32,31.56,17.823750,2.650000,17.20,85.86,52.467917,3.000000,2020-12-31,2021-01-05
6,2020-12-31 2021-01-06,8.62,32.06,18.968333,2.693333,17.67,78.20,45.423333,1.041667,2020-12-31,2021-01-06
7,2020-12-31 2021-01-07,8.89,31.27,18.899167,2.663333,20.50,78.95,50.365833,0.125000,2020-12-31,2021-01-07


In [43]:
# Separando y guardando datasets por día de reporte, reduciendo a 7 días de horizonte

for i in range(len(fechas)):
  data = pd.DataFrame()
  data = df[['slice date','t_min','t_max','temp','wind','humidity_min','humidity_max','humidity','clouds']].where(df['forecast date'] == fechas[i]).dropna()[1:8]
  data = data.rename(columns={'slice date':'date'})
  data = data.set_index('date')
  data.to_csv(RUTA+'forecast_historical/'+fechas[i]+' report.csv')